In [20]:
# !pip install fyers-apiv2 python-dotenv 
from fyers_api import fyersModel
from fyers_api import accessToken
import pandas as pd
import datetime as dt
import os
from dotenv import load_dotenv
load_dotenv("../.env")


True

In [21]:
client_id=os.environ.get('API_CLIENT_ID')
secret_key=os.environ.get('API_SECRET_KEY')
redirect_uri=os.environ.get('API_REDIRECT_URL')

In [22]:
session=accessToken.SessionModel(client_id=client_id,
secret_key=secret_key,redirect_uri=redirect_uri, 
response_type='code', grant_type='authorization_code')


In [23]:
auth_code_provider = session.generate_authcode()
print(auth_code_provider)

https://api.fyers.in/api/v2/generate-authcode?client_id=QW4JS7GAG2-100&redirect_uri=https%3A%2F%2F127.0.0.1%3A5000%2Flogin&response_type=code&state=None


In [24]:
auth_code=str(input("Enter the auth_code: "))

In [25]:
session.set_token(auth_code)
response = session.generate_token()
access_token = response["access_token"]
if(access_token is None):
    print("Access token is not generated")
else:
    print("Access token is generated")
    

Access token is generated


In [26]:
fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)
is_async = True

In [27]:
fyers.get_profile()

ERR: logEntryFunc: get_profile : [Errno 13] Permission denied: '/2022-08-08.txt'


{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'XA15265',
  'name': 'AIYAZ ANWAR QURESHI',
  'image': None,
  'display_name': None,
  'pin_change_date': '31-07-2022 00:55:39',
  'email_id': 'qureshi.aiyaz123@gmail.com',
  'pwd_change_date': '31-07-2022 00:54:23',
  'PAN': '---------',
  'pwd_to_expire': 142}}

In [28]:
# Getting Data
def dynamicDataFetching(startDate,endDate,symbol):
    # Creating a dataframe to store the data
    df = pd.DataFrame(columns=["date","open","high","low","close","volume"])
    today = dt.date.today()
    range_from=startDate
    while(range_from<=today):
    # Check whether the upcoming endDate is lesser than today if we add 99D or else make it today
        if((range_from + dt.timedelta(days=99))<today):
            range_to = range_from + dt.timedelta(days=99)
        else:
            range_to = today

        # Api call 
        data = {"symbol":symbol,"resolution":"5","date_format":"1","range_from":range_from,"range_to":range_to,"cont_flag":"1"}
        res = fyers.history(data)
        res = res["candles"]
        
        # Converting the data into pandas dataframe 
        tempDf = pd.DataFrame(columns=["date","open","high","low","close","volume"])
        for i in range(len(res)):
             tempDf.loc[i]= [
                res[i][0],
                res[i][1],
                res[i][2],
                res[i][3],
                res[i][4],
                res[i][5]]
       
        # Appending the data to the main dataframe
        df = pd.concat([df,tempDf])

        # Updating the range_from to last date of last api call
        range_from=range_to+dt.timedelta(days=1)
    return df

In [29]:
startDate = dt.date(2017,1,1)
endDate = dt.date.today()
symbol="NSE:NTPC-EQ"

df = dynamicDataFetching(startDate,endDate,symbol)
print(df.dtypes)
print(df.describe())

ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'
ERR: logEntryFunc: history : [Errno 13] Permission denied: '/2022-08-08.txt'

KeyboardInterrupt: 

In [19]:
print(df.head())

NameError: name 'df' is not defined

In [29]:
# Converting everything to playable format + changing the time zone from utc to ist
df["date"]= pd.to_datetime(df["date"],unit='s',utc=True).dt.tz_convert('Asia/Kolkata')
df.set_index("date",inplace=True)
df["open"]=df["open"].astype(float)
df["high"]=df["high"].astype(float)
df["low"]=df["low"].astype(float)
df["close"]=df["close"].astype(float)
df["volume"]=df["volume"].astype(float)

print(df.head())
print("\nData types :\n" , df.dtypes)

                             open    high     low   close    volume
date                                                               
2017-07-03 09:15:00+05:30  191.35  192.10  191.35  191.60  276376.0
2017-07-03 09:20:00+05:30  191.75  193.40  191.75  193.20  329709.0
2017-07-03 09:25:00+05:30  193.05  194.10  192.95  194.10  295047.0
2017-07-03 09:30:00+05:30  194.00  194.75  194.00  194.75  445852.0
2017-07-03 09:35:00+05:30  194.75  195.65  194.55  195.30  450285.0

Data types :
 open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object


In [26]:
# Export df to csv
fileName = "5Y_UPL.csv"
df.to_csv(f"../data/{fileName}")